In [1]:
# -*- coding: UTF-8 -*-
# © Kohulan Rajan - 2020
import argparse
import os
import pickle

import tensorflow as tf

2024-08-10 22:06:32.185885: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-10 22:06:32.187099: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-08-10 22:06:32.211979: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-08-10 22:06:32.212494: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-10 22:06:32.656299: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not fin

In [2]:
import sys

# Simulate command-line arguments (useful in interactive environments)
sys.argv = ["decimer_transformation/Utils/Create_TFrecord_From_pickles.py", "0","1"]

In [3]:
parser = argparse.ArgumentParser()
parser.add_argument("file", nargs="+")

args = parser.parse_args()
print(args)
# Initial Setup for GPU
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

gpus = tf.config.experimental.list_physical_devices("GPU")
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)


Namespace(file=['0', '1'])


2024-08-10 22:06:48.884099: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-08-10 22:06:48.885033: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [11]:
for file_in in args.file:
    train_captions = pickle.load(
    open("pkl/Capts_" + file_in + "_" + file_in + ".pkl", "rb"))
    img_name_vector = pickle.load(
        open("pkl/Images_" + file_in + "_" + file_in + ".pkl", "rb"))

    print("Total number of selected SMILES Strings: ", len(train_captions), "\n")
    # num_shards = int(len(train_captions) / 128)
    num_shards = 1 # corresponds to total train files

    file_index = num_shards * int(file_in)


Total number of selected SMILES Strings:  2 

Total number of selected SMILES Strings:  2 



In [14]:
def _bytes_feature(value):
    """Returns a bytes_list from a string / byte."""
    if isinstance(value, type(tf.constant(0))):
        value = (
            value.numpy()
        )  # BytesList won't unpack a string from an EagerTensor.
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

In [18]:
file_index

1

In [ ]:

    def get_train_tfrecord(num_shards, train_captions, img_name_vector, file_index):
        print("Total number of TFrecords: ", num_shards, flush=True)

        for i in range(num_shards):
            subsets_num = int(len(train_captions) / num_shards)
            sub_split_img_id = img_name_vector[i * subsets_num : (i + 1) * subsets_num]
            sub_split_cap_train = train_captions[
                i * subsets_num : (i + 1) * subsets_num
            ]

            tfrecord_name = "check/" + "train-%02d.tfrecord" % file_index
            writer = tf.io.TFRecordWriter(tfrecord_name)
            counter = 0
            for j in range(len(sub_split_img_id)):
                # print(decoded_image.shape)
                caption_ = sub_split_cap_train[j]
                # image_id_ = sub_split_img_id[counter]
                counter = counter + 1
                feature = {
                    # 'image_id': _bytes_feature(image_id_.encode('utf8')),
                    "image_raw": _bytes_feature(tf.io.read_file(sub_split_img_id[j])),
                    "caption": _bytes_feature(caption_.tostring()),
                }
                example = tf.train.Example(features=tf.train.Features(feature=feature))

                serialized = example.SerializeToString()
                writer.write(serialized)
            print("%s write to tfrecord success!" % tfrecord_name, flush=True)
            file_index = file_index + 1

